In [ ]:
!pip install -q -U transformers[torch]
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U bitsandbytes
!pip install -q rouge_score
!pip install -q -U bert_score
!pip install -q -U peft
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import re
import random
import numpy as np
from scipy.special import softmax
import pprint

import bitsandbytes as bnb

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig,pipeline
from transformers import TrainingArguments, Trainer

import pandas as pd
from tqdm import tqdm

# Local Data Loading Load Data

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to data save in Drive
train = 'FairytaleQA_train.csv'
valid = 'FairytaleQA_valid.csv'
test = 'FairytaleQA_test.csv'
path = 'drive/MyDrive/266_Danielle_Dylan_final_project/data/'
# path = 'drive/MyDrive/DataSci_266/266_Danielle_Dylan_final_project/data/'

In [ ]:
train_filepath = path+train
valid_filepath = path+valid
test_filepath = path+test

In [ ]:
# Save this because we'll need to tell the trainer how many examples we have
num_train_examples = 8_548

In [ ]:
train = load_dataset("csv",data_files=train_filepath,streaming=True)['train']
val = load_dataset("csv",data_files=valid_filepath,streaming=True)['train']
test = load_dataset("csv",data_files=test_filepath,streaming=True)['train']

In [ ]:
train

IterableDataset({
    features: Unknown,
    num_shards: 1
})

In [ ]:
from datasets import load_dataset, DatasetDict

# Combine into a single DatasetDict
ds = DatasetDict({
    "train": train,
    "test": test,
    "validation": val,
})

In [ ]:
train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

In [ ]:
# Initialize pipeline
model_id = "google/flan-t5-small"

In [ ]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



model = pipeline(
   "text2text-generation",
   model=model_id,
   model_kwargs={"torch_dtype": torch.bfloat16, "quantization_config": quantization_config},
   device_map="auto",
   trust_remote_code=True,
)



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
VOCAB_SIZE = 15000
MAX_SEQUENCE_LENGTH = 512

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "
context = " Context: "

# Define the preprocessing function

def preprocess_function(data):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   data['full_input'] = [context + story + prefix + question for question,story in zip(data["question"],data['story_section'])]

   input_encoded = tokenizer.batch_encode_plus(data['full_input'],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   answers_encoded = tokenizer.batch_encode_plus(data["answer1"],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   return {'input_ids': input_encoded['input_ids'],
           'labels': answers_encoded['input_ids']}

In [ ]:
def create_seq2seq_training_args(batch_size, num_epochs):

    training_args = Seq2SeqTrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_seq2seq_trainer(model, training_args, train_ds, val_ds):

    trainer = Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

In [ ]:
def create_training_args(batch_size, num_epochs):

    training_args = TrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_trainer(model, training_args, train_ds, val_ds):

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

# Fine Tuning Together

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
# Load the BLEU metric and the trained BLEURT model for semantic similarity scoring

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Map the preprocessing function across our dataset
train_ex = train.filter(lambda x: x['ex_or_im'] == 'explicit')
val_ex = val.filter(lambda x: x['ex_or_im'] == 'explicit')
test_ex = test.filter(lambda x: x['ex_or_im'] == 'explicit')

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers import AutoTokenizer

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
model_checkpoint = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 16
num_epochs = 3

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_both = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,quantization_config=quantization_config)

# for name, param in model.named_parameters():
#     param.requires_grad = False

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_both = get_peft_model(model_both, peft_config)
model_both.print_trainable_parameters()

trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [ ]:
_= model_both.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_tokenized = train.map(preprocess_function, batched=True)
val_tokenized = val.map(preprocess_function, batched=True)
test_tokenized = test.map(preprocess_function, batched=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples / batch_size),
    report_to='none'
)

In [ ]:
# train_small = train_tokenized.select(range(1000))
# val_small = val_tokenized.select(range(700))

In [ ]:
# train_small

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_both,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    #compute_metrics=compute_metrics_rouge,
)

trainer.train()

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
0,32.006300,33.125000
1,18.093800,14.250000
2,13.349200,8.289062


TrainOutput(global_step=1602, training_loss=27.63311485642946, metrics={'train_runtime': 1939.4536, 'train_samples_per_second': 13.216, 'train_steps_per_second': 0.826, 'total_flos': 4787350794141696.0, 'train_loss': 27.63311485642946, 'epoch': 2.3320848938826466})

In [ ]:
model_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_both'
model_both.save_pretrained(model_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_both'
model_both = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_filepath)

In [ ]:
def generate_output(model, tokenizer, data, **kwargs):

    all_outputs = []
    prefix = "Please answer this question: "
    context = " Context: "
    input_sentences = [context + sample['story_section'] + prefix + sample['question'] for sample in data]

    for i in range(int(len(input_sentences)) + 1):
        if i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[i], truncation=True,padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
        all_outputs.extend(generated_sentences)

    return all_outputs

In [ ]:
def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

    # Translate original texts
    answers = generate_output(model, tokenizer, data, **kwargs)

    # Calculate ROUGE scores
    rouge_results = rouge.compute(predictions=answers, references=data['answer1'])
    print('ROUGE: ', rouge_results)

    # Calculate BLEURT scores
    bleurt_scores = []
    for i in range(int(len(answers)) + 1):
        if i >= len(answers):
            break

        with torch.no_grad():
            scores = bleurt_model(**bleurt_tokenizer(data['answer1'][i],
                                                     answers[i],
                                                     truncation=True,
                                                     max_length=MAX_SEQUENCE_LENGTH,
                                                     padding='max_length',
                                                     return_tensors='pt'))[0].squeeze().numpy()
            if scores.shape:
                bleurt_scores.extend(scores)
            else:  # Happens when there was only one example in the last batch
                bleurt_scores.append(float(scores))

    print('BLEURT: ', np.mean(bleurt_scores))

    return answers

In [ ]:
# """
# Playing with the decoder .generate() arguments like num_beams or top_p, etc.
# """

# generate_kwargs = {

#     'num_beams': 4,
#     'do_sample':True,
#     'top_k':100,
#     'top_p':1,
#     'temperature':0.2,
#     'no_repeat_ngram_size':2
# }

# test_ex_answers = calculate_eval_metrics(
#     test_ex,
#     model_both,
#     tokenizer,
#     **generate_kwargs
# )

In [ ]:
pipe = pipeline('text2text-generation',model=model_both,tokenizer =tokenizer)

Device set to use cuda:0


In [ ]:
from evaluate import load
bertscore = load("bertscore")

In [ ]:
%%time

# Store results for aggregate scoring

def process_dataset(dataset):
  results = []
  bleurt_scores = []
  for idx, sample in enumerate(tqdm(dataset)):

      # if idx >= 10:  # Stop after processing 10 samples
      #   return pd.DataFrame(results)
      #   break

      prefix = "Please answer this question: "
      context = " Context: "

      question = context + sample['story_section'] + prefix + sample['question']

      # Generate summary via the pipeline
      outputs = pipe(
                          question,
                          max_new_tokens=500,
                          truncation=True
      )

      answer = outputs[0]["generated_text"]

      if answer.isspace():
        print("\nANSWER IS SPACE for question:",sample['question'],"\nReference:", sample['answer1'])
        print("\nanswer:",[answer])
        answer = 'blank'

      # print("\nbegin answer:", answer, ":end answer")
      # print("\n",[answer],"****")

      # Evaluation
      predictions = [answer]
      reference1 = [sample['answer1']]
      reference2 = [sample['answer2']]

      # Calculate ROUGE scores
      rouge_scores1 = rouge.compute(predictions=predictions, references=reference1)
      if reference1 != reference2:
        rouge_scores2 = rouge.compute(predictions=predictions, references=reference2)
      else:
        rouge_scores2 = rouge_scores1
      rouge_scores1 = {k+'_a1': v for k, v in rouge_scores1.items()}
      rouge_scores2 = {k+'_a2': v for k, v in rouge_scores2.items()}

      # Calculate BERTScore
      bertscore_result1 = bertscore.compute(predictions=predictions,
                                            references=reference1,
                                            lang="en")
      if reference1 != reference2:
        bertscore_result2 = bertscore.compute(predictions=predictions,
                                              references=reference2,
                                              lang="en")
      else:
        bertscore_result2 = bertscore_result1

      # Each BERTScore result is a list; extract first (only) value
      bertscore_f1_a1 = bertscore_result1["f1"][0]
      bertscore_f1_a2 = bertscore_result2["f1"][0]



      # Calculate BLEU scores
      bleu_scores1 = bleu.compute(predictions=predictions, references=reference1,smooth=True)
      if reference1 != reference2:
        bleu_scores2 = bleu.compute(predictions=predictions, references=reference2,smooth=True)
      else:
        bleu_scores2 = bleu_scores1
      bleu_scores1 = {k+'_a1': v for k, v in bleu_scores1.items()}
      bleu_scores2 = {k+'_a2': v for k, v in bleu_scores2.items()}

      # Calculate BLEURT scores
      with torch.no_grad():
        bleurt_score_a1 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer1'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        if reference1 != reference2:
          bleurt_score_a2 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer2'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        else:
          bleurt_score_a2 = bleurt_score_a1


      # Store results
      results.append({
          'id': idx,
          'question': sample['question'],
          'ex_or_im': sample['ex_or_im'],
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer1': sample['answer1'],
          'reference_answer2': sample['answer2'],
          'generated_answer': answer,
          **rouge_scores1,
          **rouge_scores2,
          **bleu_scores1,
          **bleu_scores2,
          'bertscore_f1_a1': bertscore_f1_a1,
          'bertscore_f1_a2': bertscore_f1_a2,
          'bleurt_score_a1': bleurt_score_a1,
          'bleurt_score_a2': bleurt_score_a2,
      })

      # Print progress update every 10 samples
      if (idx + 1) % 100 == 0:
          print(f"\nProcessed {idx + 1} samples; Results length: {len(results)}")
          print(f"Latest Answer1 ROUGE-1: {rouge_scores1['rouge1_a1']:.4f}")

  return pd.DataFrame(results)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [ ]:
test_ex_ds = Dataset.from_list(list(test_ex))
test_ex_ds

Dataset({
    features: ['story_name', 'story_section', 'question', 'answer1', 'answer2', 'local_or_sum', 'attribute', 'ex_or_im', 'ex_or_im2'],
    num_rows: 754
})

In [ ]:
test_results_ex_df = process_dataset(test_ex_ds)

  0%|          | 0/754 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

 80%|███████▉  | 600/754 [23:25<04:40,  1.82s/it]


Processed 600 samples; Results length: 600
Latest Answer1 ROUGE-1: 0.0000


 93%|█████████▎| 700/754 [26:55<02:11,  2.44s/it]


Processed 700 samples; Results length: 700
Latest Answer1 ROUGE-1: 0.1667


100%|██████████| 754/754 [28:58<00:00,  2.31s/it]


In [ ]:
test_results_ex_df.head()

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bertscore_f1_a1,bertscore_f1_a2,bleurt_score_a1,bleurt_score_a2
0,0,what did ian want to attend ?,explicit,"' stay with me , i pray , for i love company a...",a horse - race in the town .,a horse - race .,The race,0.5,0.0,0.25,...,0.169542,"[0.6666666666666666, 0.5, 1.0, 1.0]",0.223130,0.400000,2,5,0.870016,0.894993,-0.89989096,-0.9029608
1,1,what did the widowed mistress want to give to ...,explicit,""" oh , sweet madam mistress , "" she cried , "" ...",the pig .,the pig .,bacon,0.0,0.0,0.00,...,0.113803,"[0.5, 1.0, 1.0, 1.0]",0.135335,0.333333,1,3,0.837191,0.837191,-1.1056045,-1.1056045
2,2,how did andrew survive ?,explicit,but the latter did not lose heart . he gathere...,shot sea - birds .,"shot sea - birds , and looked for mussels and ...",andrew survived,0.0,0.0,0.00,...,0.004305,"[0.3333333333333333, 0.5, 1.0, 1.0]",0.006738,0.166667,2,12,0.798823,0.803281,-1.408149,-1.6678156
3,3,why did the boy keep the four-shilling piece ?,explicit,""" yes , and i 'm sure it is because you have t...",it was honest .,because it was honest .,None of the above choices,0.0,0.0,0.00,...,0.229575,"[0.16666666666666666, 0.2, 0.25, 0.33333333333...",1.000000,1.000000,5,5,0.845667,0.843429,-0.78157616,-0.9335044
4,4,"who was young, handsome, and gifted ?",explicit,old dragonbeard must have been a master swords...,tsui .,tsui .,the son,0.0,0.0,0.00,...,0.638943,"[0.3333333333333333, 0.5, 1.0, 1.0]",1.000000,1.000000,2,2,0.808486,0.808486,-1.2059941,-1.2059941


In [ ]:
test_im_ds = Dataset.from_list(list(test_im))
test_results_im_df = process_dataset(test_im_ds)

  0%|          | 0/253 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 40%|███▉      | 100/253 [04:10<06:20,  2.49s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0000


 79%|███████▉  | 200/253 [08:35<02:23,  2.70s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 253/253 [11:04<00:00,  2.63s/it]


In [ ]:
test_results_im_df.head()

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bertscore_f1_a1,bertscore_f1_a2,bleurt_score_a1,bleurt_score_a2
0,0,why did ian fall into a dark slumber ?,implicit,"' here i must leave you , ' said the deer , ' ...",he touched an enchanted stick .,the spike of hurt .,None of the above choices,0.0,0.0,0.0,...,0.302138,"[0.5, 0.2, 0.25, 0.3333333333333333]",1.000000,1.00,5,5,0.846315,0.846653,-1.361353,-1.4664459
1,1,why did the princess leave her gold ring in th...,implicit,she had run to her little room and had quickly...,to give a hint to her father about her whereab...,to tease her father .,None of the above choices,0.0,0.0,0.0,...,0.229575,"[0.16666666666666666, 0.2, 0.25, 0.33333333333...",1.000000,1.00,5,5,0.849843,0.833696,-1.2972983,-1.5650891
2,2,how did prince feel after the young men left ?,implicit,"but the herring were now ready , and the stude...",sad .,woeful .,None of the above choices,0.0,0.0,0.0,...,0.229575,"[0.16666666666666666, 0.2, 0.25, 0.33333333333...",1.000000,2.50,5,2,0.851793,0.837275,-1.3250598,-0.958719
3,3,how did the fisherman's wife feel when she hea...,implicit,"but when he got home , he found that heaven ha...",sorrowful .,"she began to weep and wail , and pray to god .",She was saddened,0.0,0.0,0.0,...,0.022494,"[0.25, 0.3333333333333333, 0.5, 1.0]",0.049787,0.25,3,12,0.848375,0.864489,-0.40637577,-0.8367876
4,4,why did the two countesses try to take off the...,implicit,alas ! alas ! scarce had he gained the hall th...,the vest would kill him .,to save him from the poison in the vest .,None of the above choices,0.2,0.0,0.2,...,0.100436,"[0.3333333333333333, 0.2, 0.25, 0.333333333333...",0.367879,0.50,5,10,0.837860,0.847705,-1.5755886,-1.5295148


In [ ]:
test_both_outputs = pd.concat([test_results_ex_df,test_results_im_df])
test_both_outputs.shape

(1007, 31)

In [ ]:
test_both_outputs.head()

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bertscore_f1_a1,bertscore_f1_a2,bleurt_score_a1,bleurt_score_a2
0,0,what did ian want to attend ?,explicit,"' stay with me , i pray , for i love company a...",a horse - race in the town .,a horse - race .,The race,0.5,0.0,0.25,...,0.169542,"[0.6666666666666666, 0.5, 1.0, 1.0]",0.223130,0.400000,2,5,0.870016,0.894993,-0.89989096,-0.9029608
1,1,what did the widowed mistress want to give to ...,explicit,""" oh , sweet madam mistress , "" she cried , "" ...",the pig .,the pig .,bacon,0.0,0.0,0.00,...,0.113803,"[0.5, 1.0, 1.0, 1.0]",0.135335,0.333333,1,3,0.837191,0.837191,-1.1056045,-1.1056045
2,2,how did andrew survive ?,explicit,but the latter did not lose heart . he gathere...,shot sea - birds .,"shot sea - birds , and looked for mussels and ...",andrew survived,0.0,0.0,0.00,...,0.004305,"[0.3333333333333333, 0.5, 1.0, 1.0]",0.006738,0.166667,2,12,0.798823,0.803281,-1.408149,-1.6678156
3,3,why did the boy keep the four-shilling piece ?,explicit,""" yes , and i 'm sure it is because you have t...",it was honest .,because it was honest .,None of the above choices,0.0,0.0,0.00,...,0.229575,"[0.16666666666666666, 0.2, 0.25, 0.33333333333...",1.000000,1.000000,5,5,0.845667,0.843429,-0.78157616,-0.9335044
4,4,"who was young, handsome, and gifted ?",explicit,old dragonbeard must have been a master swords...,tsui .,tsui .,the son,0.0,0.0,0.00,...,0.638943,"[0.3333333333333333, 0.5, 1.0, 1.0]",1.000000,1.000000,2,2,0.808486,0.808486,-1.2059941,-1.2059941


In [ ]:
test_both_outputs.to_csv('drive/MyDrive/266_Danielle_Dylan_final_project/answer_outputs/t5_trained_overall_model_both_BertScore.csv',index=False)

### Generation Experimentation

In [ ]:
# # Print out a sample of outputs to manually review
# for i in range(10):
#     sample_i = random.choice(range(len(val_ex_answers)))
#     print('Referenced answer:  ', val_ex['answer1'][i])
#     print('Generated answer:   ', val_ex_answers[i])
#     print()

In [ ]:
# """
# Playing with the decoder .generate() arguments like num_beams or top_p, etc.
# """

# generate_kwargs = {

#     'num_beams': 4,
#     'do_sample':True,
#     'top_k':100,
#     'top_p':1,
#     'temperature':0.2,
#     'no_repeat_ngram_size':2
# }

# val_im_answers = calculate_eval_metrics(
#     val_im,
#     model_both,
#     tokenizer,
#     **generate_kwargs
# )

In [ ]:
# # Print out a sample of outputs to manually review
# for i in range(10):
#     sample_i = random.choice(range(len(val_im_answers)))
#     print('Referenced answer:  ', val_im['answer1'][i])
#     print('Generated answer:   ', val_im_answers[i])
#     print()

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_both, tokenizer, trainer
clear_gpu_memory()


# Archived Work

## Original Generation DNU

In [ ]:
# Store results for aggregate scoring
results = []
for idx, sample in enumerate(tqdm(train_ex)):
    if idx >= 10:  # Stop after processing 10 samples
      break
    try:
      prefix = "Please answer this question: "
      context = " Context: "

      question = prefix + sample['question'] + context + sample['story_section']
      # Generate summary via the pipeline
      outputs = model(
                          question,
                          max_new_tokens=700,
                          num_beams=4,
                          do_sample=True,
                          top_k=100,
                          top_p=1.0,
                          temperature=0.2,
                          no_repeat_ngram_size=2
      )


      answer = outputs[0]["generated_text"]


      # Calculate ROUGE scores
      predictions = [answer]
      references = [[sample['answer1']]]
      rouge_scores = rouge.compute(predictions=predictions, references=references)


      # Store results
      results.append({
          'id': idx,
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer': sample['answer1'],
          'generated_answer': answer,
           **rouge_scores
      })

      # Print progress update every 10 samples
      if (idx + 1) % 10 == 0:
          print(f"\nProcessed {idx + 1} samples")
          print(f"Latest ROUGE-1: {rouge_scores['rouge1']:.4f}")

    except Exception as e:
      print(f"Error processing sample {idx}: {str(e)}")
      continue

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate and print average ROUGE scores
avg_scores = results_df[['rouge1', 'rouge2', 'rougeL']].mean()
print("\nAverage ROUGE Scores:")
for metric, score in avg_scores.items():
   print(f"{metric}: {score:.4f}")

# Print some example summaries
print("\nExample Summaries:")
for i in range(min(50, len(results_df))):
   print(f"\nExample {i}:")
   print(f"Reference: {results_df.iloc[i]['reference_answer']}")
   print(f"Generated: {results_df.iloc[i]['generated_answer']}")

In [ ]:
# Store results for aggregate scoring
results = []
for idx, sample in enumerate(tqdm(val_im)):
    if idx >= 10:  # Stop after processing 10 samples
      break
    try:
      prefix = "Please answer this question: "
      context = " Context: "

      question = prefix + sample['question'] + context + sample['story_section']
      # Generate summary via the pipeline
      outputs = model(
                          question,
                          max_new_tokens=700,
                          num_beams=4,
                          do_sample=True,
                          top_k=100,
                          top_p=1.0,
                          temperature=0.2,
                          no_repeat_ngram_size=2
      )


      answer = outputs[0]["generated_text"]


      # Calculate ROUGE scores
      predictions = [answer]
      references = [[sample['answer1']]]
      rouge_scores = rouge.compute(predictions=predictions, references=references)


      # Store results
      results.append({
          'id': idx,
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer': sample['answer1'],
          'generated_answer': answer,
           **rouge_scores
      })

      # Print progress update every 10 samples
      if (idx + 1) % 10 == 0:
          print(f"\nProcessed {idx + 1} samples")
          print(f"Latest ROUGE-1: {rouge_scores['rouge1']:.4f}")

    except Exception as e:
      print(f"Error processing sample {idx}: {str(e)}")
      continue

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate and print average ROUGE scores
avg_scores = results_df[['rouge1', 'rouge2', 'rougeL']].mean()
print("\nAverage ROUGE Scores:")
for metric, score in avg_scores.items():
   print(f"{metric}: {score:.4f}")

# Print some example summaries
print("\nExample Summaries:")
for i in range(min(50, len(results_df))):
   print(f"\nExample {i}:")
   print(f"Reference: {results_df.iloc[i]['reference_answer']}")
   print(f"Generated: {results_df.iloc[i]['generated_answer']}")

## Build Classification Model for Question Type

In [ ]:
checkpoint = 'google-bert/bert-base-uncased'
bert_classification_model = BertForSequenceClassification.from_pretrained(checkpoint)
bert_tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [ ]:
# Define the preprocessing function

def preprocess_classification_function(data,tokenizer):
   # The "inputs" are the tokenized answer:
   inputs = data["question"]
   model_inputs = tokenizer(text_target=inputs,
                      max_length=512,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   label_map = {"explicit": 0, "implicit": 1}
   model_inputs["labels"] = [label_map[x] for x in data["ex_or_im"]]
   return model_inputs

In [ ]:
# Map the preprocessing function across our dataset
train_clf_tokenized = train.map(preprocess_classification_function, batched=True,fn_kwargs={'tokenizer': bert_tokenizer})
val_clf_tokenized = val.map(preprocess_classification_function, batched=True,fn_kwargs={'tokenizer': bert_tokenizer})
test_clf_tokenized = test.map(preprocess_classification_function, batched=True,fn_kwargs={'tokenizer': bert_tokenizer})

train_clf_tokenized = {'input_ids': train_tokenized['input_ids'], 'labels': train_tokenized['labels']}
val_clf_tokenized = {'input_ids': val_tokenized['input_ids'], 'labels': val_tokenized['labels']}
test_clf_tokenized = {'input_ids': test_tokenized['input_ids'], 'labels': test_tokenized['labels']}

In [ ]:
metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
def fine_tune_classification_model(classification_model,
                                   tokenizer,
                                   train_data,
                                   dev_data,
                                   batch_size = 32,
                                   num_epochs = 2):
    """
    Preprocess the data using the given tokenizer (we've give you the code for that part).
    Create the training arguments and trainer for the given model and data (write your code for that).
    Then train it.
    """

    preprocessed_train_data = train_data.map(preprocess_classification_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    preprocessed_dev_data = dev_data.map(preprocess_classification_function, batched=True, fn_kwargs={'tokenizer': tokenizer})

    training_args = TrainingArguments(
        output_dir='bert_fine_tuned_clf',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        eval_strategy='epoch',
        save_strategy='epoch',
        report_to='none'
    )
    trainer = Trainer(
        model=classification_model,
        args=training_args,
        train_dataset=preprocessed_train_data,
        eval_dataset=preprocessed_dev_data,
        compute_metrics=compute_metrics
    )

    trainer.train()

In [ ]:
def fine_tune_clf_model_freeze_layers(classification_model,
                                          tokenizer,
                                          train_data,
                                          dev_data,
                                          layers_to_train = ["classifier."],
                                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                                          batch_size = 32,
                                          num_epochs = 2):
    """
    Freeze any parameters inside the given model that have a name containing one of the
    strings in the "layers_to_freeze" list.
    Then specify the training arguments and trainer for the given model and data.
    Then train it.
    """

    preprocessed_train_data = train_data.map(preprocess_classification_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    preprocessed_dev_data = dev_data.map(preprocess_classification_function, batched=True, fn_kwargs={'tokenizer': tokenizer})

    ### YOUR CODE HERE

    # freeze all layers except for the layers_to_train
    for name, param in classification_model.named_parameters():
      if not any(x in name for x in layers_to_train):
        param.requires_grad = False

    training_args = TrainingArguments(
        output_dir='bert_fine_tuned_clf',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        eval_strategy='epoch',
        save_strategy='epoch',
        report_to='none'
    )

    trainer = Trainer(
        model=classification_model,
        args=training_args,
        train_dataset=preprocessed_train_data,
        eval_dataset=preprocessed_dev_data,
        compute_metrics=compute_metrics
    )


    ### END YOUR CODE

    trainer.train()

In [ ]:
fine_tune_classification_model(bert_classification_model, bert_tokenizer, train, val)

In [ ]:
from datasets import concatenate_datasets

def downsample(train_df,verbose=False):
  '''Downsamples train_df to balance classes'''
  #balance classes in train
  explicit = train_df.filter(lambda x: x['ex_or_im'] == 'explicit')
  implicit = train_df.filter(lambda x: x['ex_or_im'] == 'implicit')

  if verbose:
      print(f"Original implicit count: {len(implicit)}")
      print(f"Original explicit count: {len(explicit)}")

  explicit = explicit.shuffle().select(range(len(implicit)))

  train_downsampled = concatenate_datasets([implicit,explicit]).shuffle()

  if verbose:
      print(f"New explicit count: {len(train_downsampled.filter(lambda x: x['ex_or_im'] == 'explicit'))}")
      print(f"New implicit count: {len(train_downsampled.filter(lambda x: x['ex_or_im'] == 'implicit'))}")

  return train_downsampled

In [ ]:
train_downsampled = downsample(train,verbose=True)

In [ ]:
layers_to_train = ['classifier.','pooler.','LayerNorm.','embeddings.']
fine_tune_clf_model_freeze_layers(bert_classification_model,bert_tokenizer,train_downsampled,val,layers_to_train)

## Try qLORA for Model Specialization

In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

In [ ]:
max([len(q) for q in train['question']])

In [ ]:
model_checkpoint = "roberta-large"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 32
num_epochs = 2

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
id2label = {
    0: "implicit",
    1: "explicit"
}

label2id = {
    "implicit": 0,
    "explicit": 1
}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)
for name, param in model.named_parameters():
    param.requires_grad = False

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
_= model.to("cuda")

In [ ]:
training_args = TrainingArguments(
    output_dir="roberta-large-lora-sequence-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    report_to='none'
)

In [ ]:
train_downsampled = downsample(train,verbose=True)

In [ ]:
# Define the preprocessing function

def preprocess_classification_function_lora(data,tokenizer):
   # The "inputs" are the tokenized answer:
   inputs = data["question"]
   model_inputs = tokenizer(text_target=inputs,
                      max_length=150,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   label_map = {"explicit": 0, "implicit": 1}
   model_inputs["labels"] = [label_map[label] for label in data["ex_or_im"]]
   return model_inputs

In [ ]:
# Map the preprocessing function across our dataset
train_clf_tokenized = train_downsampled.map(preprocess_classification_function_lora, batched=True,fn_kwargs={'tokenizer': tokenizer})
val_clf_tokenized = val.map(preprocess_classification_function_lora, batched=True,fn_kwargs={'tokenizer': tokenizer})
test_clf_tokenized = test.map(preprocess_classification_function_lora, batched=True,fn_kwargs={'tokenizer': tokenizer})

# train_clf_tokenized = {'input_ids': train_clf_tokenized['input_ids'], 'labels': train_clf_tokenized['labels']}
# val_clf_tokenized = {'input_ids': val_clf_tokenized['input_ids'], 'labels': val_clf_tokenized['labels']}
# test_clf_tokenized = {'input_ids': test_clf_tokenized['input_ids'], 'labels': test_clf_tokenized['labels']}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_clf_tokenized,
    eval_dataset=val_clf_tokenized,
    compute_metrics=compute_metrics,
)

trainer.train()

## Train Individual T-5 Lora Models on Implicit vs Explicit

In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers import AutoTokenizer

## Explicit

In [ ]:
# Save this because we'll need to tell the trainer how many examples we have
num_train_examples_ex = 6_382

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


In [ ]:
model_checkpoint = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 16
num_epochs = 3

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_ex = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,quantization_config=quantization_config)

# for name, param in model.named_parameters():
#     param.requires_grad = False

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_ex = get_peft_model(model_ex, peft_config)
model_ex.print_trainable_parameters()

In [ ]:
_= model_ex.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_ex = train.filter(lambda x: x['ex_or_im'] == 'explicit')
val_ex = val.filter(lambda x: x['ex_or_im'] == 'explicit')
test_ex = test.filter(lambda x: x['ex_or_im'] == 'explicit')

train_tokenized_ex = train_ex.map(preprocess_function, batched=True)
val_tokenized_ex = val_ex.map(preprocess_function, batched=True)
test_tokenized_ex = test_ex.map(preprocess_function, batched=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_ex / batch_size),
    report_to='none'
)

In [ ]:
# train_small = train_tokenized_ex.select(range(1000))
# val_small = val_tokenized_ex.select(range(700))

In [ ]:
# train_small

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_ex,
    args=training_args,
    train_dataset=train_tokenized_ex,
    eval_dataset=val_tokenized_ex,
    #compute_metrics=compute_metrics_rouge,
)

trainer.train()

In [ ]:
model_ex_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_ex'
model_ex.save_pretrained(model_ex_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_ex = AutoModelForSeq2SeqLM.from_pretrained(model_ex_checkpoint_filepath)

In [ ]:
def generate_output(model, tokenizer, data, **kwargs):

    all_outputs = []
    prefix = "Please answer this question: "
    context = " Context: "
    input_sentences = [prefix + sample['question'] + context + sample['story_section'] for sample in data]

    for i in range(int(len(input_sentences)) + 1):
        if i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
        all_outputs.extend(generated_sentences)

    return all_outputs

In [ ]:
def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

    # Translate original texts
    answers = generate_output(model, tokenizer, data, **kwargs)

    # Calculate ROUGE scores
    rouge_results = rouge.compute(predictions=answers, references=data['answer1'])
    print('ROUGE: ', rouge_results)

    # Calculate BLEURT scores
    bleurt_scores = []
    for i in range(int(len(answers)) + 1):
        if i >= len(answers):
            break

        with torch.no_grad():
            scores = bleurt_model(**bleurt_tokenizer(data['answer1'][i],
                                                     answers[i],
                                                     truncation=True,
                                                     max_length=MAX_SEQUENCE_LENGTH,
                                                     padding='max_length',
                                                     return_tensors='pt'))[0].squeeze().numpy()
            if scores.shape:
                bleurt_scores.extend(scores)
            else:  # Happens when there was only one example in the last batch
                bleurt_scores.append(float(scores))

    print('BLEURT: ', np.mean(bleurt_scores))

    return answers

In [ ]:
"""
Playing with the decoder .generate() arguments like num_beams or top_p, etc.
"""

generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

val_ex_answers = calculate_eval_metrics(
    val_small.select(range(100)),
    model_ex,
    tokenizer,
    **generate_kwargs
)

In [ ]:
# Print out a sample of outputs to manually review
for i in range(10):
    sample_i = random.choice(range(len(val_ex_answers)))
    print('Referenced answer:  ', val_small['answer1'][i])
    print('Generated answer:   ', val_ex_answers[i])
    print()

## Implicit

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_ex, tokenizer, trainer
clear_gpu_memory()


In [ ]:
num_train_examples_im = 2_166

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


In [ ]:
model_checkpoint = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 16
num_epochs = 3

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,quantization_config=quantization_config)

# for name, param in model.named_parameters():
#     param.requires_grad = False

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_im = get_peft_model(model_im, peft_config)
model_im.print_trainable_parameters()

In [ ]:
_= model_im.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

train_tokenized_im = train_im.map(preprocess_function, batched=True)
val_tokenized_im = val_im.map(preprocess_function, batched=True)
test_tokenized_im = test_im.map(preprocess_function, batched=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_im / batch_size),
    report_to='none'
)

In [ ]:
# train_small = train_tokenized_im.select(range(2000))
# val_small = val_tokenized_im.select(range(200))

In [ ]:
# val_small

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_im,
    args=training_args,
    train_dataset=train_tokenized_im,
    eval_dataset=val_tokenized_im,
    #compute_metrics=compute_metrics_rouge,
)

trainer.train()

In [ ]:
model_im_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_im'
model_im.save_pretrained(model_im_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_im_checkpoint_filepath)

In [ ]:
def generate_output(model, tokenizer, data, **kwargs):

    all_outputs = []
    prefix = "Please answer this question: "
    context = " Context: "
    input_sentences = [prefix + sample['question'] + context + sample['story_section'] for sample in data]

    for i in range(int(len(input_sentences)) + 1):
        if i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
        all_outputs.extend(generated_sentences)

    return all_outputs

In [ ]:
def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

    # Translate original texts
    answers = generate_output(model, tokenizer, data, **kwargs)

    # Calculate ROUGE scores
    rouge_results = rouge.compute(predictions=answers, references=data['answer1'])
    print('ROUGE: ', rouge_results)

    # Calculate BLEURT scores
    bleurt_scores = []
    for i in range(int(len(answers)) + 1):
        if i >= len(answers):
            break

        with torch.no_grad():
            scores = bleurt_model(**bleurt_tokenizer(data['answer1'][i],
                                                     answers[i],
                                                     truncation=True,
                                                     max_length=MAX_SEQUENCE_LENGTH,
                                                     padding='max_length',
                                                     return_tensors='pt'))[0].squeeze().numpy()
            if scores.shape:
                bleurt_scores.extend(scores)
            else:  # Happens when there was only one example in the last batch
                bleurt_scores.append(float(scores))

    print('BLEURT: ', np.mean(bleurt_scores))

    return answers

In [ ]:
train_small

In [ ]:
"""
Playing with the decoder .generate() arguments like num_beams or top_p, etc.
"""

generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

val_answers_im = calculate_eval_metrics(
    val_small.select(range(100)),
    model_im,
    tokenizer,
    batch_size,
    **generate_kwargs
)

In [ ]:
# Print out a sample of outputs to manually review
for i in range(10):
    sample_i = random.choice(range(len(val_answers_im)))
    print('Referenced answer:  ', val_small['answer1'][i])
    print('Generated answer:   ', val_answers_im[i])
    print()

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_im, tokenizer, trainer
clear_gpu_memory()


## Improving Implicit Answering with CoT Experiment

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "
context = " Context: "
cot_steps = """
Let's think step by step:
1. What are the key details in the story?
2. What does the question ask for?
3. How do the story details relate to the question?
4. What can we conclude?"""

# Define the preprocessing function

def preprocess_function_cot(data):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   data['full_input'] = [context + story + prefix + question + cot_steps for question,story in zip(data["question"],data['story_section'])]

   input_encoded = tokenizer.batch_encode_plus(data['full_input'],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   answers_encoded = tokenizer.batch_encode_plus(data["answer1"],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   return {'input_ids': input_encoded['input_ids'],
           'labels': answers_encoded['input_ids']}

In [ ]:
num_train_examples_im = 2_166

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


In [ ]:
model_checkpoint = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 16
num_epochs = 3

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_im_cot = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,quantization_config=quantization_config)

# for name, param in model.named_parameters():
#     param.requires_grad = False

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_im_cot = get_peft_model(model_im_cot, peft_config)
model_im_cot.print_trainable_parameters()

In [ ]:
_= model_im_cot.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

train_tokenized_im = train_im.map(preprocess_function, batched=True)
val_tokenized_im = val_im.map(preprocess_function, batched=True)
test_tokenized_im = test_im.map(preprocess_function, batched=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_im / batch_size),
    report_to='none'
)

In [ ]:
# train_small = train_tokenized_im.select(range(2000))
# val_small = val_tokenized_im.select(range(200))

In [ ]:
# val_small

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_im_cot,
    args=training_args,
    train_dataset=train_tokenized_im,
    eval_dataset=val_tokenized_im,
    #compute_metrics=compute_metrics_rouge,
)

trainer.train()

In [ ]:
model_im_cot_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_im_cot'
model_im_cot.save_pretrained(model_im_cot_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_im_cot = AutoModelForSeq2SeqLM.from_pretrained(model_im_cot_checkpoint_filepath)

In [ ]:
def generate_output(model, tokenizer, data, **kwargs):

    all_outputs = []
    prefix = "Please answer this question: "
    context = " Context: "
    cot_steps = """
    Let's think step by step:
    1. What are the key details in the story?
    2. What does the question ask for?
    3. How do the story details relate to the question?
    4. What can we conclude?"""

    input_sentences = [context + sample['story_section'] + prefix + sample['question'] +cot_steps for sample in data]

    for i in range(int(len(input_sentences)) + 1):
        if i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
        all_outputs.extend(generated_sentences)

    return all_outputs

In [ ]:
def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

    # Translate original texts
    answers = generate_output(model, tokenizer, data, **kwargs)
    references = [example['answer1'] for example in data]

    # Calculate ROUGE scores
    rouge_results = rouge.compute(predictions=answers, references=references)
    print('ROUGE: ', rouge_results)

    # Calculate BLEURT scores
    bleurt_scores = []
    for i in range(int(len(answers)) + 1):
        if i >= len(answers):
            break

        with torch.no_grad():
            scores = bleurt_model(**bleurt_tokenizer(references[i],
                                                     answers[i],
                                                     truncation=True,
                                                     max_length=MAX_SEQUENCE_LENGTH,
                                                     padding='max_length',
                                                     return_tensors='pt'))[0].squeeze().numpy()
            if scores.shape:
                bleurt_scores.extend(scores)
            else:  # Happens when there was only one example in the last batch
                bleurt_scores.append(float(scores))

    print('BLEURT: ', np.mean(bleurt_scores))

    return answers

In [ ]:
# train_small

In [ ]:
"""
Playing with the decoder .generate() arguments like num_beams or top_p, etc.
"""

generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':200,
    'top_p':0.95,
    'temperature':0.4,
    'no_repeat_ngram_size':4,
    'max_new_tokens': 500
}

test_im_list = list(test_im.take(100))  # or however many you evaluated

test_answers_im = calculate_eval_metrics(
    # test_tokenized_im,
    test_im_list,
    model_im_cot,
    tokenizer,
    batch_size,
    **generate_kwargs
)

In [ ]:
# Print out a sample of outputs to manually review

for i in range(10):
    sample_i = random.choice(range(len(test_im_list)))
    print('Referenced answer:  ', test_im_list[i]['answer1'])
    print('Generated answer:   ', test_answers_im[i])
    print()

In [ ]:
test_im_list[0]

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_im_cot, tokenizer, trainer
clear_gpu_memory()


## Trying Llama Family on Implicit questions

In [ ]:
num_train_examples_im = 2_166

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


In [ ]:
# model_checkpoint = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_checkpoint = 'Qwen/Qwen2-1.5B-Instruct'
model_checkpoint = 'Qwen/Qwen2-0.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 8
num_epochs = 3

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_im_llama = AutoModelForCausalLM.from_pretrained(model_checkpoint,quantization_config=quantization_config,
                                                      device_map='auto')

# for name, param in model.named_parameters():
#     param.requires_grad = False

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_im_llama = get_peft_model(model_im_llama, peft_config)
model_im_llama.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [ ]:
_= model_im_llama.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

train_tokenized_im = train_im.map(preprocess_function, batched=True)
val_tokenized_im = val_im.map(preprocess_function, batched=True)
test_tokenized_im = test_im.map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_im / batch_size),
    report_to='none'
)

In [ ]:
# train_small = train_tokenized_im.select(range(2000))
# val_small = val_tokenized_im.select(range(200))

In [ ]:
# val_small

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer = Trainer(
    model=model_im_llama,
    args=training_args,
    train_dataset=train_tokenized_im,
    eval_dataset=val_tokenized_im,
    #compute_metrics=compute_metrics_rouge,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,0.334600,0.257977
1,0.182500,0.129631
2,0.141800,0.123261


TrainOutput(global_step=812, training_loss=1.6050995566648216, metrics={'train_runtime': 2496.4166, 'train_samples_per_second': 2.602, 'train_steps_per_second': 0.325, 'total_flos': 7148496647356416.0, 'train_loss': 1.6050995566648216, 'epoch': 2.332512315270936})

In [ ]:
model_im_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/qwen_model_im'
model_im_llama.save_pretrained(model_im_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_im_llama = AutoModelForCausalLM.from_pretrained(model_im_checkpoint_filepath)

In [ ]:
def generate_output(model, tokenizer, data, **kwargs):

    all_outputs = []
    prefix = "Please answer this question: "
    context = " Context: "

    input_sentences = [prefix + sample['question'] + context + sample['story_section'] for sample in data]

    for i in range(int(len(input_sentences)) + 1):
        if i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)

        all_outputs.extend(generated_sentences)

    return all_outputs

In [ ]:
def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

    # Translate original texts
    answers = generate_output(model, tokenizer, data, **kwargs)
    references = [example['answer1'] for example in data]

    # Calculate ROUGE scores
    rouge_results = rouge.compute(predictions=answers, references=references)
    print('ROUGE: ', rouge_results)

    # Calculate BLEURT scores
    bleurt_scores = []
    for i in range(int(len(answers)) + 1):
        if i >= len(answers):
            break

        with torch.no_grad():
            scores = bleurt_model(**bleurt_tokenizer(references[i],
                                                     answers[i],
                                                     truncation=True,
                                                     max_length=MAX_SEQUENCE_LENGTH,
                                                     padding='max_length',
                                                     return_tensors='pt'))[0].squeeze().numpy()
            if scores.shape:
                bleurt_scores.extend(scores)
            else:  # Happens when there was only one example in the last batch
                bleurt_scores.append(float(scores))

    print('BLEURT: ', np.mean(bleurt_scores))

    return answers

In [ ]:
"""
Playing with the decoder .generate() arguments like num_beams or top_p, etc.
"""

generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':200,
    'top_p':0.95,
    'temperature':0.4,
    'no_repeat_ngram_size':4,
    'max_new_tokens': 500
}

test_im_list = list(test_im.take(100))  # or however many you evaluated

test_answers_im = calculate_eval_metrics(
    # test_tokenized_im,
    test_im_list,
    model_im_llama,
    tokenizer,
    batch_size,
    **generate_kwargs
)

ROUGE:  {'rouge1': np.float64(0.03323503692703145), 'rouge2': np.float64(0.010509581911600122), 'rougeL': np.float64(0.029018620394766888), 'rougeLsum': np.float64(0.029103070410378122)}
BLEURT:  -1.9081339573860168


In [ ]:
# Print out a sample of outputs to manually review

for i in range(10):
    sample_i = random.choice(range(len(test_im_list)))
    print('Referenced answer:  ', test_im_list[i]['answer1'])
    print('Generated answer:   ', test_answers_im[i])
    print()

Referenced answer:   he heard the weathercocks .
Generated answer:    Please answer this question: why did matte sit up at midnight ? Context: about midnight the fisherman sat up , and said to his wife : ' dost thou hear anything ? ' ' no , ' said she . ' i think the twirling of the weathercock on the roof bodes ill , ' said he ; ' we shall have a storm . ' ' oh , it is nothing but your fancy , ' said his wife . matte lay down , but soon rose again . ' the weathercock is squeaking now , ' said he . ' just fancy ! go to sleep , ' said his wife ; and the old man tried to . for the third time he jumped out of bed . ' ho ! how the weather - cock is roaring at the pitch of its voice , as if it had a fire inside it ! we are going to have a tempest , and must bring in the net . '

Referenced answer:   he will meet snorro .
Generated answer:    Please answer this question: what will harold do after he rides away on his horse ? Context: now , when the countess fraukirk had been away upon her wi

In [ ]:
test_im_list[2]

{'story_name': 'four-shilling-piece',
 'story_section': 'now when the merchant had sailed on for a while , a terrible storm broke loose , a thunderstorm without an equal , and he drifted and drifted , and did not know where or whither . at last he came to a land where he had never yet been , and went up into the city . in the tavern which he entered the table was set , and at every place lay a switch , one for each guest . this seemed strange to the merchant , for he could not understand what was to be done with all the switches . yet he sat down and thought : " i will watch carefully , and see just what the rest do with them , and then i can imitate them . " yes , and when the food came on the table , then he knew why the switches were there : the place was alive with thousands of mice , and all who were sitting at the table had to work and fight and beat about them with their switches , and nothing could be heard but the slapping of the switches , one worse than the other . sometimes

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_im_cot, tokenizer, trainer
clear_gpu_memory()
